[View in Colaboratory](https://colab.research.google.com/github/aksh98/FintechProject/blob/master/OES_Fintech.ipynb)

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [9]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0


Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1buil

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

In [33]:
%cd drive
%cd Colab Notebooks/FintechProject
# !ls -l | wc -l
# %cd .
!ls

[Errno 2] No such file or directory: 'drive'
/content/drive/Colab Notebooks/FintechProject
[Errno 2] No such file or directory: 'Colab Notebooks/FintechProject'
/content/drive/Colab Notebooks/FintechProject
OES_00_17  OES_2_00_17	OES_Fintech.ipynb


In [13]:
!pip install xlrd

import xlrd
import os


In [34]:
## CODE

path = "/content/drive/Colab Notebooks/FintechProject/OES_2_00_17/"
files = os.listdir(path)

for i in files:
  if(i == ".DS_Store"):
    continue
  else:
    #print workspaces
#     print(i)
    # Open workbook --- 
    path_sh = str(path)+str(i)

#     print(path_sh)
    workbook = xlrd.open_workbook(path_sh)
    worksheet = workbook.sheet_by_index(0)
    
    # data extraction => 4,2 - worksheet.cell(4,2).value
    
    num_rows = worksheet.nrows
    num_cols = worksheet.ncols
    print(i,"           Size of the sheet: ",num_rows,num_cols)
# for rows 
    

('natsector_M2007_dl.xls', '           Size of the sheet: ', 8736, 24)
('natsector_M2011_dl.xlsx', '           Size of the sheet: ', 8438, 24)
('nat2d_sic_2001.xls', '           Size of the sheet: ', 16747, 24)
('natsector_M2017_dl.xlsx', '           Size of the sheet: ', 16156, 24)
('natsector_M2016_dl.xlsx', '           Size of the sheet: ', 16201, 24)
('natsector_M2015_dl.xlsx', '           Size of the sheet: ', 16101, 24)
('natsector_M2014_dl.xlsx', '           Size of the sheet: ', 15954, 24)
('natsector_M2013_dl.xls', '           Size of the sheet: ', 15922, 24)
('natsector_M2012_dl.xls', '           Size of the sheet: ', 15833, 24)
('natsector_M2011_dl.xls', '           Size of the sheet: ', 8438, 25)
('natsector_M2010_dl.xls', '           Size of the sheet: ', 8439, 24)
('natsector_M2009_dl.xls', '           Size of the sheet: ', 8534, 24)
('natsector_M2008_dl.xls', '           Size of the sheet: ', 8662, 24)
('natsector_M2006_dl.xls', '           Size of the sheet: ', 8755, 24

In [0]:
# READ THE SHEET AND 
# ---- store it in a table- --------

path_sh = str(path)+str('natsector_M2017_dl.xlsx')

workbook = xlrd.open_workbook(path_sh)
worksheet = workbook.sheet_by_index(0)
table = []
num_rows = worksheet.nrows 
num_cols = worksheet.ncols


for x in range(num_rows):
  record = []
  for y in range(num_cols):      
    record.append(worksheet.cell(x,y).value)
#   record.append('0')
  table.append(record)
  


# *  = indicates that a wage estimate is not available    
# # = indicates a wage that is equal to or greater than $100.00 per hour or $208,000 per year    
# ** = indicates that an employment estimate is not available
# ~ = indicates that the percent of establishments reporting the occupation is less than 0.5%


# --------^^^-------
# for x in range(num_rows):
#   record = []
#   for y in range(num_cols):      
#     if(worksheet.cell(x,y).value == '*'):
#       record.append('')
#     if(worksheet.cell(x,y).value == '**'):
#       record.append('')
#     if(worksheet.cell(x,y).value == '#' and y > 15 and y < 21): #a
#       record.append('')
#     if(worksheet.cell(x,y).value == '#' and y > 10 and y < 16): #h
#       record.append('')
#     if(worksheet.cell(x,y).value == '~'):
#       record.append('')
# # --------^^^-------
#     else:   
#       record.append(worksheet.cell(x,y).value)
  
#   record.append('0')
#   table.append(record)
  
  

In [36]:
for i in range(0,2):
  print(table[i])
print(int(table[2][0])/10000 == int(table[5][0])/10000)
print(len(table),len(table[0]))

[u'NAICS', u'NAICS_TITLE', u'OCC_CODE', u'OCC_TITLE', u'OCC_GROUP', u'TOT_EMP', u'EMP_PRSE', u'PCT_TOTAL', u'PCT_RPT', u'H_MEAN', u'A_MEAN', u'MEAN_PRSE', u'H_PCT10', u'H_PCT25', u'H_MEDIAN', u'H_PCT75', u'H_PCT90', u'A_PCT10', u'A_PCT25', u'A_MEDIAN', u'A_PCT75', u'A_PCT90', u'ANNUAL', u'HOURLY']
[u'11', u'Agriculture, Forestry, Fishing and Hunting', u'00-0000', u'Industry Total', u'total', 424020.0, 1.1, 100.0, 100.0, 14.89, 30960.0, 0.7, 10.13, 10.57, 11.77, 16.12, 23.35, 21070.0, 21980.0, 24480.0, 33530.0, 48570.0, u'', u'']
True
(16156, 24)


In [0]:
# reduce by: 10000
naic = 10000
for x in range(1,num_rows):
  for i in range(x,num_rows):
    
    if(table[x][22] == '0'):
      if(int(table[x][0])/naic == int(table[i][0])/naic):
        if(table[x][2] == table[i][2]):
          table[x][4] = int(table[x][4]) + int(table[i][4])
          for j in range(4,num_cols-1):
            table[x][j] = (int(table[x][j])*int(table[x][22]) + int(table[i][5]))/(int(table[x][22])+1)
            table[x][22] = int(table[x][22]) + 1
            

In [0]:
## -------------------------------------------------------------------------
## -------------------------------------------------------------------------

import numpy as np

# from bokeh.layouts import gridplot
# from bokeh.plotting import figure, show, output_file

!pip install missingno
# !pip install quilt
# !quilt install ResidentMario/missingno_data

In [37]:
import missingno as msno
import pandas as pd

df = pd.DataFrame(table)
df.info()
print(table)
table = table.nyc_collision_factors()
table = table.replace("*",np.nan)
table = table.replace("**",np.nan)
table = table.replace("#",np.nan)
table = table.replace("~",np.nan)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



AttributeError: ignored

In [38]:
!NotebookApp.iopub_data_rate_limit=10000000000

[I 17:41:13.704 NotebookApp] Serving notebooks from local directory: /content/drive/Colab Notebooks/FintechProject
[I 17:41:13.704 NotebookApp] 0 active kernels
[I 17:41:13.704 NotebookApp] The Jupyter Notebook is running at:
[I 17:41:13.704 NotebookApp] http://localhost:8888/?token=1f50635bd687a4454c21b7b28216cda008719d8a01a137f2
[I 17:41:13.704 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[W 17:41:13.705 NotebookApp] No web browser found: could not locate runnable browser.
[C 17:41:13.705 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://localhost:8888/?token=1f50635bd687a4454c21b7b28216cda008719d8a01a137f2
^C
[I 17:51:21.349 NotebookApp] interrupted
Serving notebooks from local directory: /content/drive/Colab Notebooks/FintechProject
0 active kernels
The Jupyter Notebook is running at:
http://localhost:8888/?token=1f50635bd687a4454c21b7b

In [41]:
!jupyter notebook --generate-config

Writing default config to: /content/.jupyter/jupyter_notebook_config.py


In [46]:
# %cd ../../../..
%cd /content/.jupyter/

/content/.jupyter


In [47]:
!ls

jupyter_notebook_config.py  migrated


In [0]:
!jupyter notebook --generate-config
open("jupyter_notebook_config.py",'rw')

Overwrite /content/.jupyter/jupyter_notebook_config.py with default config? [y/N]